In [112]:
"""
D17 과제
- 이 체인은 영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답해야 합니다. (예: "탑건" -> "🛩️👨‍✈️🔥". "대부" -> "👨‍👨‍👦🔫🍝").
- 앞서 배운 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현합니다.
- 항상 세 개의 이모티콘으로 답장하도록 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하여 체인에 예시를 제공하세요.
- 메모리가 작동하는지 확인하려면 체인에 두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청하세요.

작업 순서
1. memory 없이 먼저 구현. d16 기준
2. 강의 듣고, memory 구현

"""

'\nD17 과제\n- 이 체인은 영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답해야 합니다. (예: "탑건" -> "🛩️👨\u200d✈️🔥". "대부" -> "👨\u200d👨\u200d👦🔫🍝").\n- 앞서 배운 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현합니다.\n- 항상 세 개의 이모티콘으로 답장하도록 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하여 체인에 예시를 제공하세요.\n- 메모리가 작동하는지 확인하려면 체인에 두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청하세요.\n\n작업 순서\n1. memory 없이 먼저 구현. d16 기준\n2. 강의 듣고, memory 구현\n\n'

In [113]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.prompts import MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough


chat = ChatOpenAI(
    temperature=0.5,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_token_limit=1000,
    return_messages=True,
    memory_key="chat_history",
)

examples = [
    {
        "movie": "스타워즈",
        "answer": """
        🌌⚔️👽

        - 🌌: 스타워즈는 우주 공간을 배경으로 한 환상적인 세계를 보여주는 영화이기 때문에 은은한 우주의 느낌을 담은 이모티콘을 선택했습니다.

        - ⚔️: 영화에서 빛나는 레이저 검이나 전투 장면이 많이 등장하는데, 이를 상징하는 이모티콘으로 스타워즈의 전투적인 요소를 표현했습니다.

    
        - 👽: 스타워즈는 다양한 외계 생명체들이 등장하는데, 그 중에서도 다양한 외계인들이 등장하여 영화의 다양성을 나타내기 때문에 외계 생명체를 상징하는 이모티콘을 선택했습니다.
        """,
    },
    {
        "movie": "탑건",
        "answer": """
        🚀🔥😎
        
        🚀: '탑건'은 전투기 조종사들의 역동적인 비행씬과 스릴 넘치는 전투 장면으로 인해 마치 하늘을 나는 듯한 느낌을 주는 영화입니다. 전투기가 속도를 내며 하늘을 가로지르는 장면은 마치 로켓처럼 빠르고 감동적입니다.

        🔥: 영화 속 주인공들의 열정과 용기가 불길처럼 뜨거운 에너지를 전달해줍니다. 전투기 조종사들이 마주하는 위험과 도전 속에서도 그들의 열정과 용기가 불을 지펴내는 듯한 느낌을 줍니다.

        😎: '탑건'은 스타일리시한 비행복과 섹시한 룩으로 유명한 영화로, 주인공들의 카리스마 넘치는 매력이 관객을 사로잡습니다. 영화 속 주인공들은 마치 쿨한 선장처럼 매 순간을 멋지게 소화해내는 것이 인상적입니다.
        """,
    },
    {
        "movie": "반지의 제왕",
        "answer": """
        👑🧝‍♂️🔥

        👑: '반지의 제왕'은 왕좌를 차지할 수 있는 힘을 상징하는 반지를 중심으로 전쟁과 운명의 이야기를 그린 영화로, 왕권과 권력에 대한 욕망을 상징하는 왕관을 상징합니다.

        🧝‍♂️: 영화 속에 등장하는 엘프들은 우아하고 아름다운 존재로, 자연과 조화를 이루며 숲을 수호하는 역할을 합니다. 엘프는 영화의 신비로움과 순수함을 상징합니다.

        🔥: 영화는 전투와 결투로 가득 차 있으며, 화려한 전투씬은 영화의 긴장감과 스릴을 높여줍니다. 불꽃같이 타오르는 전투는 캐릭터들의 용기와 희생을 상징합니다.        
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            "영화 {movie} 에 대해서 이모티콘 3개로 표현해줘. 그리고 그 이유에 대해서 알려줘.",
        ),
        (
            "ai",
            "{answer}",
        ),
    ]
)
example_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 영화의 내용을 {style} 표현하는 것으로 유명한 {role} 입니다. ",
        ),
        example_prompt,
        MessagesPlaceholder(variable_name="chat_history"),
        (
            "human",
            "영화 {movie} 에 대해서 이모티콘 3개로 표현해줘. 그리고 그 이유에 대해서 알려줘.",
        ),
        # ("ai", "{answer}"),
    ]
)

general_prompt = ChatPromptTemplate.from_messages(
    [
        example_prompt,
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]


movie_chain = RunnablePassthrough.assign(chat_history=load_memory) | final_prompt | chat
general_chain = (
    RunnablePassthrough.assign(chat_history=load_memory) | general_prompt | chat
)


def invoke_movie_chain(movie):
    result = movie_chain.invoke(
        {
            "role": "웹툰 작가",
            "style": "심미적으로",
            "movie": movie,
        }
    )
    memory.save_context(
        {"input": movie},
        {"output": result.content},
    )


def invoke_general_chain(question):
    result = general_chain.invoke(
        {
            "question": question,
        }
    )
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )


invoke_movie_chain("인생은 아름다워")


        🎨👨‍👦‍👦😊

        🎨: '인생은 아름다워'는 인생의 다양한 색채와 아름다움을 담아내는 영화로, 삶의 다양한 면면을 아름답게 그려냅니다. 또한, 영화 속 아름다운 풍경과 예술적인 장면들이 마치 화가의 그림처럼 아름다움을 상징합니다.

        👨‍👦‍👦: 영화는 아버지와 아들 간의 따뜻한 이야기를 중심으로 전개됩니다. 가족의 소중함과 사랑을 다루며, 아버지와 아들의 소중한 시간을 함께 보내는 모습을 통해 가족의 소중함을 상징합니다.

        😊: '인생은 아름다워'는 삶의 소소한 행복과 감동을 담은 영화로, 주인공의 긍정적인 마인드와 주변 사람들과의 소중한 인연을 통해 행복과 감사함을 느끼게 합니다. 따뜻한 감정과 행복한 마음을 상징하는 이모티콘을 선택했습니다.        
        

In [114]:
invoke_movie_chain("고질라")

🦖🔥🌆

- 🦖: '고질라'는 거대한 괴물인 고질라를 상징하는 이모티콘입니다. 영화 속 고질라는 거대하고 강력한 존재로, 인류에게 위협을 주는 존재로 그려집니다.

- 🔥: 영화는 거대한 괴물과 인류의 대결을 다루는데, 고질라의 불꽃방사와 거대한 파괴력이 도시를 불태우는 장면은 화려하고 강렬한 느낌을 줍니다.

- 🌆: '고질라'는 도시를 파괴하는 장면이 많이 등장하는데, 이를 상징하는 이모티콘입니다. 거대한 고질라와 도시의 대결은 도시의 취약성과 인류의 힘 없이는 존재할 수 없음을 보여줍니다.

In [115]:
memory.load_memory_variables({})["chat_history"]

[HumanMessage(content='인생은 아름다워'),
 AIMessage(content="\n        🎨👨\u200d👦\u200d👦😊\n\n        🎨: '인생은 아름다워'는 인생의 다양한 색채와 아름다움을 담아내는 영화로, 삶의 다양한 면면을 아름답게 그려냅니다. 또한, 영화 속 아름다운 풍경과 예술적인 장면들이 마치 화가의 그림처럼 아름다움을 상징합니다.\n\n        👨\u200d👦\u200d👦: 영화는 아버지와 아들 간의 따뜻한 이야기를 중심으로 전개됩니다. 가족의 소중함과 사랑을 다루며, 아버지와 아들의 소중한 시간을 함께 보내는 모습을 통해 가족의 소중함을 상징합니다.\n\n        😊: '인생은 아름다워'는 삶의 소소한 행복과 감동을 담은 영화로, 주인공의 긍정적인 마인드와 주변 사람들과의 소중한 인연을 통해 행복과 감사함을 느끼게 합니다. 따뜻한 감정과 행복한 마음을 상징하는 이모티콘을 선택했습니다.        \n        "),
 HumanMessage(content='고질라'),
 AIMessage(content="🦖🔥🌆\n\n- 🦖: '고질라'는 거대한 괴물인 고질라를 상징하는 이모티콘입니다. 영화 속 고질라는 거대하고 강력한 존재로, 인류에게 위협을 주는 존재로 그려집니다.\n\n- 🔥: 영화는 거대한 괴물과 인류의 대결을 다루는데, 고질라의 불꽃방사와 거대한 파괴력이 도시를 불태우는 장면은 화려하고 강렬한 느낌을 줍니다.\n\n- 🌆: '고질라'는 도시를 파괴하는 장면이 많이 등장하는데, 이를 상징하는 이모티콘입니다. 거대한 고질라와 도시의 대결은 도시의 취약성과 인류의 힘 없이는 존재할 수 없음을 보여줍니다.")]

In [129]:
invoke_general_chain("고질라 보다 먼저 물어본 영화 제목이 무엇인가요?")

고질라 보다 먼저 물어보신 영화 제목은 '반지의 제왕'이었습니다.

In [117]:
invoke_general_chain("너가 궁금한 영화 하나 골라서 이모티콘 3개로 평가해줘")

"인터스텔라"에 대해 이모티콘 3개로 평가해드리겠습니다.

🌌🚀🤯

- 🌌: '인터스텔라'는 우주 여행을 다룬 영화로, 우주의 신비로움과 끝없는 우주 공간을 표현하는 데 이모티콘을 선택했습니다.

- 🚀: 영화는 우주선을 타고 우주 여행을 떠나는 모험을 다루는데, 빠르고 역동적인 우주 비행씬을 상징하는 이모티콘을 선택했습니다.

- 🤯: '인터스텔라'는 과학적인 요소와 철학적인 내용을 다루며, 관객들에게 깊은 생각을 하게 만드는 영화로, 놀라움과 감동을 느끼게 하는 이모티콘을 선택했습니다.The human expresses that life is beautiful. The AI responds with a description of a movie called "Life is Beautiful" which portrays the beauty and various colors of life. The movie focuses on the heartwarming story between a father and son, highlighting the importance of family and love. It also conveys the message of finding happiness and gratitude in life's simple joys.

In [122]:
invoke_general_chain("예시로 제공한 목록 알려줘")

예시로 제공한 목록은 다음과 같습니다:
- 스타워즈
- 탑건
- 반지의 제왕
- Life is Beautiful
- 고질라
- 인터스텔라

위 목록은 이전 대화에서 사용된 영화 제목들입니다. 추가적인 정보나 도움이 필요하시면 언제든지 말씀해주세요.The human expresses that life is beautiful. The AI responds with a description of a movie called "Life is Beautiful" which portrays the beauty and various colors of life, focusing on the heartwarming story between a father and son. The human then mentions "Godzilla" and the AI responds with emojis symbolizing Godzilla as a giant, destructive monster threatening humanity. The movie depicts intense battles between Godzilla and humans, with scenes of fiery destruction in cities. Godzilla's encounters with cities symbolize vulnerability and the necessity of human strength to combat such threats.

In [119]:
load_memory(_)

[SystemMessage(content='The human expresses that life is beautiful. The AI responds with a description of a movie called "Life is Beautiful" which portrays the beauty and various colors of life. The movie focuses on the heartwarming story between a father and son, highlighting the importance of family and love. It also conveys the message of finding happiness and gratitude in life\'s simple joys.'),
 HumanMessage(content='고질라'),
 AIMessage(content="🦖🔥🌆\n\n- 🦖: '고질라'는 거대한 괴물인 고질라를 상징하는 이모티콘입니다. 영화 속 고질라는 거대하고 강력한 존재로, 인류에게 위협을 주는 존재로 그려집니다.\n\n- 🔥: 영화는 거대한 괴물과 인류의 대결을 다루는데, 고질라의 불꽃방사와 거대한 파괴력이 도시를 불태우는 장면은 화려하고 강렬한 느낌을 줍니다.\n\n- 🌆: '고질라'는 도시를 파괴하는 장면이 많이 등장하는데, 이를 상징하는 이모티콘입니다. 거대한 고질라와 도시의 대결은 도시의 취약성과 인류의 힘 없이는 존재할 수 없음을 보여줍니다."),
 HumanMessage(content='마지막 바로 전에 물어본 영화 제목이 무엇인가요?'),
 AIMessage(content="마지막으로 물어보신 영화 제목은 '고질라'였습니다."),
 HumanMessage(content='너가 궁금한 영화 하나 골라서 이모티콘 3개로 평가해줘'),
 AIMessage(content='"인터스텔라"에 대해 이모티콘 3개로 평가해드리겠습니다.\n\n🌌🚀🤯\n\n- 🌌: \'인터스텔라\'는 우주 여행을 다룬 영화로, 우주